<a href="https://colab.research.google.com/github/RafaelAndrade55/Aula_4_Alura/blob/main/previsao_decoque.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt

def prever_decoque(arquivo, T_limite):
  """
  Preve a necessidade de decoque com base em dados de temperatura dos tubos de saída.

  Args:
    arquivo (str): Caminho para o arquivo Excel com os dados de temperatura.
    T_limite (float): Temperatura limite que indica a necessidade de decoque (°C).

  Returns:
    None: Imprime as previsões e gera gráficos.
  """

  # Carregar os dados do arquivo Excel
  df = pd.read_excel(arquivo)

  # Remover linhas com 'Run (days)' igual a 0 (inicio de campanha)
  df = df[df['Run (days)'] != 0]

  # Calcular a taxa média de aumento de temperatura para cada tubo
  taxas_aumento_tubo = {}
  for coluna in df.columns[4:]:  # Iterar pelas colunas de temperatura (TMT#1 - TMT#54)
    tubo = coluna.split('#')[1]

    # Criar modelo de regressão linear
    modelo = LinearRegression()

    # Extrair dados de temperatura e dias de campanha, ignorando valores 0
    x = df['Run (days)'][df[coluna] != 0].values.reshape(-1, 1)
    y = df[coluna][df[coluna] != 0].values.reshape(-1, 1)

    # Treinar o modelo
    modelo.fit(x, y)

    # Armazenar a taxa de aumento (coeficiente angular da reta)
    taxas_aumento_tubo[tubo] = modelo.coef_[0][0]

  # Calcular a taxa média de aumento de temperatura global do forno
  taxa_aumento_global = np.mean(list(taxas_aumento_tubo.values()))

  # Previsão da duração da campanha para cada tubo
  print("Previsão de Duração da Campanha por Tubo:")
  print("------------------------------------------")
  for tubo, taxa in taxas_aumento_tubo.items():
    T_inicial = df[f'TMT#{tubo}'].iloc[-1]  # Temperatura mais recente do tubo
    if T_inicial != 0:
      t_campanha = (T_limite - T_inicial) / taxa
      print(f"Tubo #{tubo}: {t_campanha:.1f} dias")

  # Calcular a duração média da campanha do forno
  duracao_media_campanha = (T_limite - df.iloc[:, 4:].mean().mean()) / taxa_aumento_global
  print("\nDuração Média da Campanha do Forno:", f"{duracao_media_campanha:.1f} dias")

  # Plotar gráfico da temperatura dos tubos em função do tempo
  plt.figure(figsize=(12, 6))
  for coluna in df.columns[4:]:
    plt.plot(df['Run (days)'], df[coluna], label=coluna)
  plt.xlabel('Dias de Campanha')
  plt.ylabel('Temperatura (°C)')
  plt.title('Temperatura dos Tubos de Saída ao Longo do Tempo')
  plt.legend(loc='upper left', bbox_to_anchor=(1, 1))
  plt.grid(True)
  plt.show()

  # Salvar as taxas de aumento de temperatura em um arquivo CSV
  with open('taxas_aumento_temperatura.csv', 'w') as f:
    f.write("Tubo,Taxa de Aumento (°C/dia)\n")
    for tubo, taxa in taxas_aumento_tubo.items():
      f.write(f"{tubo},{taxa:.2f}\n")

# Solicitar o caminho do arquivo Excel para o usuário
arquivo = input("Digite o caminho para o arquivo Excel: ")

# Solicitar a temperatura limite para o usuário
T_limite = float(input("Digite a temperatura limite de decoque (°C): "))

# Executar a previsão de decoque
prever_decoque(arquivo, T_limite)

print("\nResultados salvos em 'taxas_aumento_temperatura.csv' e gráfico gerado.")



Digite o caminho para o arquivo Excel: d:\Users\rafael.andrade\Documents\Apresentações\EPC_2024\Analises_Resultados\Pirometria
Digite a temperatura limite de decoque (°C): 1075


FileNotFoundError: [Errno 2] No such file or directory: 'd:\\Users\\rafael.andrade\\Documents\\Apresentações\\EPC_2024\\Analises_Resultados\\Pirometria'